In [ ]:
import bilby
import numpy as np
from scipy import signal
import math
import seaborn as sns

import h5py 
from pathlib import Path

import matplotlib.pyplot as plt

def mod_sinusoid(time, A_0, A_am, A_fm, omega_0, omega_ax, phi_c, phi_ax):
    
    amplitude = A_0 + A_am * np.cos(1 * omega_ax * time - phi_ax)
    
    omega_c = omega_0 + A_fm * np.sin(1 * omega_ax * time - phi_ax - math.pi)
    
    phase = omega_c * time - phi_c
    
    return amplitude * np.exp(1j * phase) * np.exp(-1j * omega_lo * time)

def select_index(meta_dict, theta, rad, energy):
    
    #unique = [np.sort(np.unique(meta_dict[key])) for i, key in enumerate(meta_dict)]
    #grids = np.meshgrid(*unique)
    
    keys = list(meta_dict.keys())
    
    mask = abs(meta_dict[keys[0]] - theta) < 1e-6
    mask = np.logical_and(mask, abs(meta_dict[keys[1]] - rad) < 1e-6)
    mask = np.logical_and(mask, abs(meta_dict[keys[2]] - energy) < 1e-6)
    
    return np.argwhere(mask).squeeze()


datapath = Path.home()/'group'/'project'/'datasets'/'data'/'220414_dl_grid_84to90_1to45mm.h5'

h5file = h5py.File(datapath, 'r')

meta_keys = ['theta_min', 'x_min', 'energy']

meta_dict = {}
for i, key in enumerate(meta_keys):
    meta_dict[key] = h5file['meta'][key][:]


    
example_timeseries = h5file['data'][
    select_index(meta_dict, 88.0, 0.01, 18550)
]

example_timeseries = example_timeseries[0:1*8192]




In [ ]:
example_spectrum = np.fft.fftshift(np.fft.fft(example_timeseries, norm='forward'))
frequencies = np.fft.fftshift(np.fft.fftfreq(example_timeseries.size, 1/205e6))

peaks = signal.find_peaks(
    abs(example_spectrum),
    5e-10,
)
peak_freq = frequencies[peaks[0]]

print(np.diff(peak_freq))
print(peak_freq)

In [ ]:
sns.set_theme(context='talk', style='whitegrid', palette='bright')

f_sample = 205e6
#f_LO = 25.86e9
N_sample = 8192 * 1
time = np.arange(0, N_sample, 1) / f_sample

A_0 = 5e-9
A_am = 5e-9
A_fm = 4e4

omega_0 = 2 * math.pi * 58.e6
omega_ax = 2 * math.pi * 25e6

phi_c = 0
phi_ax = 0

model = mod_sinusoid(time, A_0, A_am, A_fm, omega_0, omega_ax, phi_c, phi_ax)

noise_var = 1.38e-23 * f_sample * 50 * 5e-4

rng = np.random.default_rng()

noise = np.random.multivariate_normal([0, 0], np.eye(2) * noise_var / 2, N_sample)
noise = noise[:, 0] + 1j * noise[:, 1]

#model = signal.hilbert(model)
#model_I = model
#model_Q = 1j * model

model_spectrum = np.fft.fftshift(np.fft.fft(model + noise, norm='forward'))
#model_spectrum = np.fft.fft(model, norm='forward')

example_spectrum = np.fft.fftshift(np.fft.fft(example_timeseries + noise, norm='forward'))
frequencies = np.fft.fftshift(np.fft.fftfreq(example_timeseries.size, 1/205e6))

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(frequencies, abs(example_spectrum))
ax.plot(frequencies, abs(model_spectrum))